In [53]:
# Milan Kovacs - 21308128, Caoimhe Cahill - 21331308
# The code runs to execution

"""
Work to be done:
    - Loading of data    # Caoimhe
    - Processing of data # Caoimhe
    - Define optimiser   # Milan
    - Define loss function # Caoimhe
    - Define activation function (sigmoid vs others) # Milan
    - Train
        - Save # Milan
        - Load # Milan
    - Add comments explaining code
    - Playing with Hyperparameters and noting the results
        - Matplotlib 
            - Training graph
            - Accuracy
    - Evaluation 
"""

In [54]:
# Imports
from typing import Callable, List
import numpy as np


In [55]:
# Constructor
class MultiLayerPerceptron:
    def __init__(self, 
                 layers: List[int], 
                 provided_loss_function: Callable[['MultiLayerPerceptron'], None] = print, # todo: decide whether we pass these functions or call a switch case
                 provided_optimiser: Callable[['MultiLayerPerceptron'], None] = print,
                 training_data: List[any] = None,
                 test_data: List[any] = None, 
                 learning_rate: float = 0.2, 
                 epoch: int = 1_000,  
                 k_fold: int = 6, 
                 split_data: bool = False) -> None:
        # Declarations
        if test_data is None:
            test_data = []
            
        if training_data is None:
            training_data = []
        
        self.layers  = layers
        self.num_layers = len(self.layers)
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.training_data = training_data
        self.test_data  = test_data
        self.k_fold = k_fold
        self.config_data = split_data
        self.provided_loss_function = provided_loss_function
        self.provided_optimiser = provided_optimiser
        self.weights = []
        self.biases  = []
        
        # Setting up
        self.set_weights()
        self.set_biases()
        
        # Transform data


        # Completion
        print(str(self))
    
    """
        * Generates a list of array shapes for each weight that corresponds to each weight in a given layer.
        * for example: if our layer is [4, 5, 2, 1], then we will have two lists to zip through
        * [4, 5, 2] and [5, 2, 1] which means on first iteration, we have our input layer of [4] nodes connecting to each [5] nodes 
        * in the first hidden layer. This will generate a shape of 5 lists, with 4 weights each.
        * Each shape corresponds to a node within our hidden layer (of 5 nodes) and in each shape the index 
        * corresponds to the index of our input layer. This is applied for the rest of the layers. 
    """
    def set_weights(self) -> None:
        layers_after_input_layer = self.layers[1:]
        layers_before_output_layer = self.layers[:-1]
        self.weights = [
            np.random.randn(y, x)
            for x, y in zip(layers_before_output_layer, layers_after_input_layer)
        ]
    
    """
        * Generates a list of Y by 1 dimensional list, where Y is the given layer size.
        * This means we can correspond each index to a layer of biases,
        * and within it, we can correspond each bias to a given node inside of that layer
    """
    def set_biases(self) -> None:
        layers_after_input = self.layers[1:]
        self.biases = [
            np.random.randn(y, 1) 
            for y in layers_after_input
        ]
        
    def loss_function(self) -> None:
        self.provided_loss_function(self)
    
    def optimiser(self) -> None:
        self.provided_optimiser(self)
    
    
        
    def __str__(self) -> str:
        return "Network generated with the following layers: " + str(self.layers)

def tmp(mlp : 'MultiLayerPerceptron') -> None:
    print(mlp.layers)
    
sample = MultiLayerPerceptron([4, 5, 2, 1], tmp, tmp)


Network generated with the following layers: [4, 5, 2, 1]


In [56]:
# Optimisers
def SGD(mlp: 'MultiLayerPerceptron'):
    pass

def Adam(mlp: 'MultiLayerPerceptron'):
    pass

In [57]:
# Activation functions
# http://neuralnetworksanddeeplearning.com/chap1.html 
# ~ Michael Nielsen 2019
threshold = 0

def sigmoid(y: float):
    return 1.0 / (1.0 + np.exp(-y))

def sigmoid_prime(y: float):
    return sigmoid(y) * (1 - sigmoid(y))
    
def stepper(y: float):
    return 1 if y >= threshold else 0

def tanh(y: float):
    return np.tanh(y)

def rectified_linear_unit(y: float):
    pass

In [58]:
# K-Fold for training and testing data
print(sample.weights)

[array([[-0.06441589, -1.15161631,  0.27548651, -1.17744553],
       [-0.32535514,  0.88569849, -0.70556203,  1.35398409],
       [ 1.19501848,  0.23627874,  0.53398499,  0.52027195],
       [ 0.91753499,  0.50238585,  2.79688044, -1.32857794],
       [ 1.23716913, -1.50503263,  0.97132252,  0.50741844]]), array([[ 0.21953692, -1.92219909, -0.26394012,  0.34961028, -0.79384531],
       [ 1.05760206, -0.46561446, -0.66923213, -0.20769766, -1.28698735]]), array([[-0.01474267,  0.1816729 ]])]
